In [ ]:
# Import all the necessary libraries
import numpy as np
import pandas as pd
import networkx as nx
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import ast
from scipy.stats import gaussian_kde
from scipy.optimize import curve_fit

import datetime

In [ ]:
# Function for saving a dataframe as a CSV 
def save_dataframe_as_csv(dataframe):
    dataframe_csv = dataframe.name + ".csv"
    dataframe.to_csv(dataframe_csv, index=False)
    return

# Processing kaggle dataset and creates a dataframe with the intended date range 
def create_subset_datasets_from_date_range(date_range_lower, date_range_upper):
    df = pd.read_csv('sp500_stocks.csv')
    if date_range_lower and date_range_upper:
        df = df[(df['Date'] >= date_range_lower) & (df['Date'] < date_range_upper)]
        df = df.dropna()
        df.reset_index(drop=True, inplace=True)
        return df

# Pass in a date and duration to get date range according to set duration
def create_date_range_from_date(date, duration):
    if date and duration:
        diff = datetime.timedelta(days = 1)
        d = datetime.timedelta(days = duration)
        datetime_before_upper = str(date - diff).split(' ')[0]
        datetime_before_lower = str(date - diff - d).split(' ')[0]
        datetime_after_lower = str(date + diff).split(' ')[0]
        datetime_after_upper = str(date + diff + d).split(' ')[0]
        
        csv_name_before = "stockMarketDataBefore_" + str(date).split(' ')[0] + ".csv"
        csv_name_after = "stockMarketDataAfter_" + str(date).split(' ')[0] + ".csv"
        df_before = create_subset_datasets_from_date_range(datetime_before_lower, datetime_before_upper)
        df_after = create_subset_datasets_from_date_range(datetime_after_lower, datetime_after_upper)
        return df_before, df_after

# Create a dataframe for the ri(t) of a stock 
def create_daily_log_closing_price(df):
    daily_log_closing_price = pd.DataFrame(columns = df['Date'].unique())
    # Drop the first column as it won't apply in the formula
    daily_log_closing_price2 = daily_log_closing_price.iloc[:,1:]
    daily_log_closing_price2.insert(0, column='Symbol', value = df['Symbol'].unique())
    daily_log_closing_price2 = daily_log_closing_price2.fillna(np.float64(0))
    return daily_log_closing_price2

# Create a dataframe for each stock: log(t) - log(t-1)
def create_daily_log_price_all(df, daily_log_closing_price):
    average_log_price_all = pd.DataFrame(columns = ['Symbol', 'Average'])
    initial_date = df.iloc[0]['Date']
    end_date = df.iloc[-1]['Date']
    for i, j in df.iterrows():
        if j['Date'] == initial_date:
            continue
        else:
            daily_log_closing_price.loc[daily_log_closing_price['Symbol'] == j['Symbol'],j['Date']] = (j['Close'] - df.loc[i-1]['Close'])
    return

# Create coefficient correlation dataframe
def create_create_coefficient_correlation_dataframe(daily_log_closing_price):
    daily_log_closing_price_No_Symbols = daily_log_closing_price.drop('Symbol',axis = 1)
    daily_log_closing_price_No_Symbols.to_numpy()
    CorrelationCoefficient = np.corrcoef(daily_log_closing_price_No_Symbols)
    CorrelationCoefficientDF = pd.DataFrame(CorrelationCoefficient, columns = daily_log_closing_price['Symbol'])
    CorrelationCoefficientDF.insert(0, column='Symbol', value = daily_log_closing_price['Symbol'].unique())
    CorrelationCoefficientDF = add_sector_data_to_dataframe(CorrelationCoefficientDF)
    return CorrelationCoefficientDF

# Add sector information to the dataframe
def add_sector_data_to_dataframe(dataframe):
    sectionDF = pd.read_csv("constituents_csv.csv")
    dataframe.insert(1,"Sector",np.nan)
    dataframe['Sector'] = dataframe["Symbol"].map(sectionDF.set_index('Symbol')['Sector'])
    return dataframe

#Create a graph based on the coefficient matrix given, threshold, or if the information given is a distance value
def create_graph(matrix, threshold, distance):
    Companies_df = pd.read_csv('sp500_companies.csv')
    G = nx.Graph()
    
    added_stocks = []
    for index, row in matrix.iterrows():
        if row['Symbol'] not in added_stocks:
            added_stocks.append(row['Symbol'])
            G.add_node(row['Symbol'], sector = str(row['Sector']), marketcap = Companies_df.loc[Companies_df['Symbol'] == row['Symbol']]['Marketcap'].item())
        for colIndex, value in enumerate(row):
            if row['Symbol'] == row.index[colIndex] or row.index[colIndex] == 'Symbol' or row.index[colIndex] == 'Sector':
                continue
            if row.index[colIndex] not in added_stocks:
                added_stocks.append(row.index[colIndex])
                try:
                    tempMarketcap = Companies_df.loc[Companies_df['Symbol'] == row.index[colIndex]]['Marketcap'].item()
                except ValueError:
                    tempMarketcap = np.nan
                G.add_node(row.index[colIndex], sector = str(matrix.loc[matrix['Symbol'] == row.index[colIndex]]['Sector'].item()), marketcap = tempMarketcap)
            if threshold:
                if value >= threshold:
                    G.add_edge(row['Symbol'], row.index[colIndex], correlation = value)
            if distance:
                if not np.isnan(value):
                    G.add_edge(row['Symbol'], row.index[colIndex], distance = value)
    return G

#Plot graph
def plot_graph(G, graph_name):
    fig = plt.figure(1,figsize=(40,40))
    pos = nx.spring_layout(G, k=0.7, iterations=50)
    nx.draw(G, pos, with_labels=True)
    graph_name = graph_name + ".jpg"
    plt.savefig(graph_name, format="JPG")

#create a graph based on a certain date range
def create_graph_for_date_range(date_range_lower, date_range_upper, threshold, graph_name="CorrelationCoGraph"):
    df = create_subset_datasets_from_date_range(date_range_lower, date_range_upper)
    df['Close'] = np.log(df['Close'])
    daily_log_closing_price = create_daily_log_closing_price(df)
    create_daily_log_price_all(df, daily_log_closing_price)
    G = create_graph_with_daily_log_closing_price(daily_log_closing_price, threshold, graph_name)
    return G

#create 2 dataframes that give you the random of dates before and after for x duration of days
def create_dataframes_specific_before_after(year, month, day, duration=40):
    datetime_test = datetime.datetime(year, month, day)
    duration = duration
    df_before, df_after = create_date_range_from_date(datetime_test, duration)
    df_before['Close'] = np.log(df_before['Close'])
    df_after['Close'] = np.log(df_after['Close'])
    daily_log_closing_price_before = create_daily_log_closing_price(df_before)
    create_daily_log_price_all(df_before, daily_log_closing_price_before)
    daily_log_closing_price_after = create_daily_log_closing_price(df_after)
    create_daily_log_price_all(df_after, daily_log_closing_price_after)
    return  daily_log_closing_price_before, daily_log_closing_price_after

#create the graph with the daily log closing price
def create_graph_with_daily_log_closing_price(daily_log_closing_price, threshold, graph_name="Graph", min_span=False):
    CorrelationCo = create_create_coefficient_correlation_dataframe(daily_log_closing_price)
    #CorrelationCo.name = "CorrelationCo"
    #save_dataframe_as_csv(CorrelationCo)
    G = create_graph(CorrelationCo, threshold, None)
    plot_graph(G, graph_name)
    return G

#Transform our function into a distance
def distance_function(x):
    return np.sqrt(2*(1-x))

#Create a datafame with distance formula - originally used when we were going to create 
def create_distance_correlation_coefficient_matrix(CorrelationCo):
    distanceDF = CorrelationCo.copy()
    distanceDF = distanceDF.loc[:, distanceDF.columns != 'Symbol'].applymap(distance_function)
    distanceDF.insert(0,"Symbol",CorrelationCo["Symbol"],True)
    return distanceDF

In [ ]:
## Degreee Distribution 
## Code used from CPSC 572 course materials: TA materials

# probability of finding an edge connecting a j degree node and k degree node
# k degree node * j degree node 
def plot_ejk(list_of_edges_bw_nodes):
    
    fig = plt.figure()
    plt.gca().invert_yaxis()
    ax = plt.gca()
    ax.set_facecolor('black')
    ax.xaxis.tick_top()
    x, y = list(), list()
    
    for edges_bw_nodes in list_of_edges_bw_nodes:
        x.append(edges_bw_nodes[0])
        y.append(edges_bw_nodes[1])

    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)

    plt.scatter(x, y, norm=colors.LogNorm(vmin=z.min(), vmax=z.max()), c=z, s=1, cmap='afmhot')
    plt.colorbar()
    
    plt.xlabel("k")
    plt.ylabel("j")
    ax_ = ax.twinx()
    plt.ylabel('e_jk')
    
    plt.show()
    
#Plot a Knn graph, modified to allow fo different sample sizes
def plot_knn(k, knn_k, SampleNum=50):
    
    k=np.array(k)
    avg_k = np.average(k)
    fig = plt.figure()
    ax = plt.gca()
    
    # Binning using a Log Scale
    bin_edges = np.logspace(0, np.log10(k.max()), num=SampleNum)
    k_nn, _ = np.histogram(knn_k, bins=bin_edges, density=True)
    log_be = np.log10(bin_edges)
    k = 10**((log_be[1:] + log_be[:-1])/2)
    ax.loglog(k, k_nn, marker='o', linestyle='none', label="ANND for the original Graph")
    
    # Random Network's Average Next Neighbor Degree
    neutral_net_knn = np.full(k.shape, np.average(np.square(k)))/avg_k
    ax.plot(k, neutral_net_knn, label="ANND for the random Graph")
    
    # Fitting k_nn = a*(k^u) to the scatter plot to get the value of u
    def func(k, a, u):
        return a*(k**u)
    popt, _ = curve_fit(func, k, k_nn)
    a, u = popt
    print("Value of u is: ", u)
    ax.plot(k, func(k, *popt), label="Value of u = {0}".format(u))
    
    plt.xlabel("k")
    plt.ylabel("k_nn")
    
    plt.show()
    

#Plot a degree distribution
def plot_degree_distribution(graph):
    list_of_edges_bw_nodes = list(nx.node_degree_xy(graph))
    plot_ejk(list_of_edges_bw_nodes)
    return

#Calculate average degree
def calculate_average_degree(G):
    N = len(G)
    L = G.size()
    return 2*L/N

#Plot degree distribution scale
def plot_degree_distribution_scale(G):
    degrees = [G.degree(node) for node in G]
    kmin = min(degrees)
    kmax = max(degrees)
    # Get 10 logarithmically spaced bins between kmin and kmax
    if kmin>0:
        bin_edges = np.logspace(np.log10(kmin), np.log10(kmax), num=20)
    else:
        bin_edges = np.logspace(0, np.log10(kmax), num=20)
    # histogram the data into these bins
    density, _ = np.histogram(degrees, bins=bin_edges, density=True)
    fig = plt.figure(figsize=(6,4))

    # "x" should be midpoint (IN LOG SPACE) of each bin
    log_be = np.log10(bin_edges)
    x = 10**((log_be[1:] + log_be[:-1])/2)
    plt.loglog(x, density, marker='o', linestyle='none')
    plt.xlabel(r"degree $k$", fontsize=16)
    plt.ylabel(r"$P(k)$", fontsize=16)
    
    a, b = np.polyfit(x, density, 1)
    plt.plot(x, a*x+b)

    # remove right and top boundaries because they're ugly
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    # Show the plot
    plt.show()
    return

# Calculate the average next neighbor degree
def plot_k_nearest_neighbours(G, sampleNum=50):
    node_and_degrees = list(G.degree)

    k = list()
    knn_k = list()

    for node, degree in node_and_degrees:
        degrees_of_neighbors = [G.degree(neighbor) for neighbor in list(G.neighbors(node))]
        if degree > 0:
            avg_neighbor_degree = sum(degrees_of_neighbors)/degree
        else:
            avg_neighbor_degree = 0

    #   print(node, degree, avg_neighbor_degree)
        k.append(degree)
        knn_k.append(avg_neighbor_degree)

    plot_knn(k, knn_k, sampleNum)
    return 

#Create a singular ER network
def create_comparative_random_network(G):
    NB_NODES = len(G.nodes())
    NB_EDGES = len(G.edges())
    P = (2 * NB_EDGES) / ((NB_NODES)*(NB_NODES-1))
    # Creating a Erdos-Renyi Network
    R = nx.erdos_renyi_graph(NB_NODES, P)
    return R

# Run ensemble of ER networks
# From lecture Null Models, modified to only consider connected components 
def run_ensemble_ER_graphs(G):
    clustering_ER = []
    short_path_ER = []
    degree_ER = []
    degree_co_ER = []
    
    GN = len(G.nodes())
    max_L = GN*(GN-1)/2
    actual_L = len(G.edges())
    p = actual_L/max_L

    for i in range(1000): # 1000 is better
        ER = nx.erdos_renyi_graph(GN, p, directed=False)
        C_ER = np.mean(list(nx.clustering(ER).values()))
        tempArr = []
        for g in (ER.subgraph(c) for c in nx.connected_components(ER)):
            if len(g.edges):
                tempArr.append(nx.average_shortest_path_length(g))
        degree_co_ER.append(nx.degree_pearson_correlation_coefficient(ER))
        degree_ER.append(calculate_average_degree(ER))
        clustering_ER.append(C_ER)
        short_path_ER.append(np.mean(tempArr))
    return clustering_ER, short_path_ER, degree_ER, degree_co_ER

# From lecture Null Models
def plot_box_plot(plotValues ,value):
    fig = plt.figure(figsize=(6,4))

    plt.boxplot(plotValues)
    plt.plot(2,value,'r',marker='+',markersize=15)
    ax = plt.gca()
    ax.set_xticks([1,2],labels=[1,2])
    plt.xlim([0.5,2.5])
    plt.show()

In [ ]:
# Run ensemble of DP graphs
# From lecture Null Models
def run_ensemble_DP_graphs(G):
    DP = G.copy()
    clustering_DP = []
    short_path_DP = []
    degree_DP = []
    degree_co_DP = []

    for i in range(1000): # 1000 is better

        nx.double_edge_swap(DP,nswap=10*G.number_of_edges(),max_tries=100000)
        C_DP = np.mean(list(nx.clustering(DP).values()))
        tempArr = []
        for g in (DP.subgraph(c) for c in nx.connected_components(DP)):
            if len(g.edges):
                tempArr.append(nx.average_shortest_path_length(g))
        degree_co_DP.append(nx.degree_pearson_correlation_coefficient(DP))
        degree_DP.append(calculate_average_degree(DP))
        clustering_DP.append(C_DP)
        short_path_DP.append(np.mean(tempArr))
    
    return clustering_DP, short_path_DP, degree_DP, degree_co_DP

In [ ]:
#Parameters: Date_Range_Lower, Date_Range_Upper, threshold, graph_name (optional)
G = create_graph_for_date_range('2020-01-01', '2022-11-01', 0.75)

In [ ]:
print(G)

In [ ]:
nx.write_gml(G, "CorrelationCovid75" + ".gml")

In [ ]:
#Parameters: year, month, day, duration (optional)
#df_before, df_after = create_dataframes_specific_before_after(2021,1,22)
#create_graph_with_daily_log_closing_price(df_before, 0.75)

In [ ]:
#create_graph_with_daily_log_closing_price(df_after, 0.75)

In [ ]:
random_network = create_comparative_random_network(G)
print(random_network)

In [ ]:
clustering_ER, short_path_ER, degree_ER, degree_co_ER = run_ensemble_ER_graphs(G)

In [ ]:
C = nx.average_clustering(G)
arr = []
for g in (G.subgraph(c) for c in nx.connected_components(G)):
    if len(g.edges):
        arr.append(nx.average_shortest_path_length(g))
d = np.mean(arr)

In [ ]:
print("Average Clustering Coefficient: ", C)
print("Average Clustering Coefficient for 1000 ER: ", np.mean(clustering_ER))
print("Standard deviation for clustering for 1000 ER: ", np.std(clustering_ER))
print("-------------------------------------------")
print("Average for path length for connected components: " + str(d))
print("Average Path length (Connected Componenets) for 1000 ER: ", np.mean(short_path_ER))
print("Standard deviation for shortest path length (Connected Componenets) for 1000 ER: ", np.std(short_path_ER))
print("-------------------------------------------")
print("Average Degree: ", calculate_average_degree(G))
print("Average Degree for 1000 ER: ", np.mean(degree_ER))
print("Standard deviation for Degree for 1000 ER: ", np.std(degree_ER))
print("Degree Correlation Coefficient for the Original Graph is: ", nx.degree_pearson_correlation_coefficient(G))
print("Average Degree Correlation Coefficient for 1000 ER: ", np.mean(degree_co_ER))

In [ ]:
plot_box_plot(clustering_ER, C)
plot_box_plot(short_path_ER, d)

In [ ]:
clustering_DP, short_path_DP, degree_DP, degree_co_DP = run_ensemble_DP_graphs(G)

In [ ]:
print("Average Clustering Coefficient: ", C)
print("Average Clustering Coefficient for 1000 ER: ", np.mean(clustering_DP))
print("Standard deviation for clustering for 1000 ER: ", np.std(clustering_DP))
print("-------------------------------------------")
print("Average for path length for connected components: " + str(d))
print("Average Path length (Connected Componenets) for 1000 ER: ", np.mean(short_path_DP))
print("Standard deviation for shortest path length (Connected Componenets) for 1000 ER: ", np.std(short_path_DP))
print("-------------------------------------------")
print("Average Degree: ", calculate_average_degree(G))
print("Average Degree for 1000 ER: ", np.mean(degree_DP))
print("Standard deviation for Degree for 1000 ER: ", np.std(degree_DP))
print("Degree Correlation Coefficient for the Original Graph is: ", nx.degree_pearson_correlation_coefficient(G))
print("Average Degree Correlation Coefficient for 1000 DP: ", np.mean(degree_co_DP))

In [ ]:
plot_box_plot(clustering_DP, C)
plot_box_plot(short_path_DP, d)

In [ ]:
plot_degree_distribution(G)

In [ ]:
plot_degree_distribution(random_network)

In [ ]:
calculate_average_degree(G)

In [ ]:
plot_degree_distribution_scale(G)

In [ ]:
plot_degree_distribution_scale(random_network)

In [ ]:
calculate_average_degree(random_network)

In [ ]:
plot_k_nearest_neighbours(G,10)